In [33]:
import json

# Initialize an empty array to store the parsed JSON objects
parsed_data = []
routes_done = []

with open('routes_done', 'r') as f:
    for line in f:
        routes_done.append(line.strip()) 

# Open and read the file line by line
try:
    with open('even_less_finer_10_to_25', 'r') as file:
        for line in file:
            try:
                # Parse each line as a JSON object and append to the array
                json_obj = json.loads(line.strip())
                parsed_data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON in line: {e}")
                continue
    
    print(f"Successfully parsed {len(parsed_data)} JSON objects from the file.")
except FileNotFoundError:
    print("File 'even_less_finer_10_to_25' not found.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")


Successfully parsed 18185 JSON objects from the file.


In [34]:
# get the best matching polylines data for each route
from geopy.distance import geodesic
routesPolylines={}
closestToStart = {}
for json_data in parsed_data:
    for deviceId in json_data:
        deviceData = json_data[deviceId]
        for routeId in deviceData:
            if routeId in routes_done:
                continue
            routeData = deviceData[routeId]
            routeFirstStop = routeData.get('stops', [])[0]
            routeLastStop = routeData.get('stops', [])[-1]
            trailFirstPoint = routeData.get('trail_segment', [])[0]
            oldList = routeData.get('trail_segment', [])
            routeData['trail_segment'] = [routeFirstStop] + oldList + [routeLastStop]
            distanceFromRouteStart = geodesic((trailFirstPoint['lat'], trailFirstPoint['long']), (routeFirstStop['lat'], routeFirstStop['long'])).meters
            if routeId in routesPolylines:
                olderValue = routesPolylines[routeId]
                if routeData['match_percentage'] == olderValue['match_percentage'] and distanceFromRouteStart < closestToStart[routeId]:
                # if True:
                    # routesPolylines[routeId].append(routeData)
                    routesPolylines[routeId] = routeData
                    closestToStart[routeId] = distanceFromRouteStart
            else:
                # routesPolylines[routeId] = [routeData]
                routesPolylines[routeId] = routeData
                closestToStart[routeId] = distanceFromRouteStart


In [35]:
with open("route_pollines_even_less_fine.json", "w") as f:
    f.write(json.dumps(routesPolylines, indent=2))
    

In [36]:
# Generate Google polylines for each route's trail segment
import polyline

# Create a hashmap to store the encoded polylines for each route
route_encoded_polylines = {}

for route_id, route_data in routesPolylines.items():
    # Extract trail segment coordinates
    trail_segment = route_data.get('trail_segment', [])
    
    # Format coordinates for polyline encoding (lat, lng format)
    coordinates = []
    for point in trail_segment:
        if 'lat' in point and 'long' in point:
            coordinates.append((point['lat'], point['long']))
    
    # Encode the coordinates as a Google polyline
    if coordinates:
        encoded_polyline = polyline.encode(coordinates)
        route_encoded_polylines[route_id] = encoded_polyline
    else:
        print("failed")

print(f"Generated polylines for {len(route_encoded_polylines)} routes")

# Save the encoded polylines to a file
with open("route_encoded_polylines.json", "w") as f:
    f.write(json.dumps(route_encoded_polylines, indent=2))

print("Encoded polylines saved to route_encoded_polylines.json")


Generated polylines for 40 routes
Encoded polylines saved to route_encoded_polylines.json


In [37]:
# Generate SQL update queries for atlas_app.route table
sql_queries = []

for route_id, encoded_polyline in route_encoded_polylines.items():
    # Create SQL update query
    sql_query = f"UPDATE atlas_app.route SET polyline = '{encoded_polyline}' WHERE code = '{route_id}' and polyline = '';"
    sql_queries.append(sql_query)

# Save the SQL queries to a file
with open("route_polyline_updates.sql", "w") as f:
    for query in sql_queries:
        f.write(query + "\n")


print(f"Generated {len(sql_queries)} SQL update queries")
print("SQL queries saved to route_polyline_updates.sql")


Generated 40 SQL update queries
SQL queries saved to route_polyline_updates.sql
